# Liquid Handling with the Syringe Tool 
This example notebook will show you how to do basic liquid handing with a syringe tool

### Before Starting:
Clear any existing items off the bed of your machine!

## Zero the Extruder Position
Our syringe tool doesn't have any limit switches; for liquid handling, that meens the syringe plunger should start at 0 mL when the machine starts. If your syringe is not already there, you can manually move to 0 and restart your machine. 

To do so, open Duet Web Control and pickup the syringe. Then, send 'extrude' commands in small increments until the plunger is at 0. Park the syringe, and restart the machine.

In [2]:
# Import the required modules
from science_jubilee.Machine import Machine
from science_jubilee.tools.Syringe import Syringe

In [ ]:
# Connect to the machine
m = Machine(address='192.168.1.2')

In [ ]:
# m.home_all() # if you need to, uncomment this line to home the machine

### Load the Deck configuration
Each lab automation deck requires calibration on a machine. Load in the calibration file like this:

In [ ]:
deck = m.load_deck('POSE-calibrated-deck')

### Load the Syringe tool

In [ ]:
# To load the tool, specify the index of the tool on your machine
# If you're not sure what your tool index is, you can query Duet about what tools are configured like this:
m.configured_tools

In [ ]:
# Change the following tool_index variable to match your machine configuration, if necessary
tool_index = 2

In [ ]:
# The syringe config file contains calibration data
# You can name the syringe whatever you'd like!
syringe = Syringe(index = tool_index,
                  name = 'syringe',
                  config = '10cc_syringe_liquidhandling'
                 )
m.load_tool(syringe)

In [ ]:
# Now you can pick up the syringe!
m.pickup_tool(syringe)

### Zero the Z Position
By default, we leave quite a bit of distance between the bed and the syringe's '0' position because we can attach tips of different lengths to the syringe. To start, we need to calibrate the syringe tip to the bed.

If you haven't calibrated the Z position yet, **do so now using the [Set Z Offset notebook](./0_SetZOffset.ipynb)**. You need to do this each time you turn the machine on.

Once you've calibrated your Z Offset, proceed with this notebook!

### Prep Labware

For this example, we'll use a petri dish as a reservoir, and move liquid into a 24 well plate.

You should add some water to a petri dish, and have your water reservoir as well as a 24 well plate at hand!

### Load Labware
Once you have gathered your materials, load the labware.

In [ ]:
# Specify the slot to install your labware
samples = m.load_labware("greiner_24_wellplate_3300ul", 0)
reservoir = m.load_labware("generic_petri_dish_100ml", 1)

# A petri dish has only 1 "well", so we can directly access it for convenience
reservoir = reservoir["A1"]

In [ ]:
# Now physically install the labware on the machine!
# You can drop the bed down to make room
m.move_to(z=150)

### Moving liquid with the transfer
We can transfer liquid with the transfer command! To transfer liquid, we specify a quantity of liquid, source well, and destination well. 

In [ ]:
# Transfer 1 mL of water to well A1 of our sample plate
sample_well = samples["A1"]
syringe.transfer(vol = 1,
                 source = reservoir,
                 destination = sample_well)

In [ ]:
# We can mix after transferring with the mix_after command
# Where we can specify (quantity_to_mix_with, number_of_mixes)
sample_well = samples["A2"]
syringe.transfer(vol = 1,
                 source = reservoir,
                 destination = sample_well,
                 mix_after = (1,2),
                )

In [ ]:
# We can also provide a list of destination wells
# For example, to fill all the wells in row C:

destination_wells = list(samples.row_data["C"])
syringe.transfer(vol = 1,
                 source = reservoir,
                 destination = destination_wells,
                 mix_after = (1,2),
                )
m.move_to(z=150) # drop the bed when finished

In [ ]:
m.move_to(z=150) # drop the bed when finished